In [15]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

In [88]:
df = pd.read_csv('../user_data_csv/pytorch_pytorch.csv')
df.head()

,username,fullName,bio,email,repository_count,company,avatar_url,isHireable,star_time,followers,...,isDeveloperProgramMember,isSiteAdmin,isViewer,anyPinnableItems,viewerIsFollowing,sponsors,primary_language,yearsofExperience,location,domainofExpertise
0,alexbw,Alex Wiltschko,NaN,alex.bw@gmail.com,47,Google,https://avatars.githubusercontent.com/u/161935...,False,2016-08-15 18:39:55,396,...,False,False,True,False,0,"{'C': 5, 'Objective-C': 5, 'C++': 1, 'Python':...",9,"Boston, MA",Data Science,NaN
1,iassael,Yannis Assael,Research Scientist,iassael@gmail.com,27,DeepMind,https://avatars.githubusercontent.com/u/216211...,True,2016-08-15 19:14:32,265,...,False,False,True,False,0,"{'JavaScript': 1, 'Java': 2, 'PHP': 1, 'C': 1,...",3,London,Data Science,NaN
2,bshillingford,Brendan Shillingford,NaN,NaN,24,NaN,https://avatars.githubusercontent.com/u/232674...,False,2016-08-16 06:35:51,446,...,False,False,True,False,0,"{'Python': 14, 'Lua': 7, 'CSS': 1, 'HTML': 1, ...",5,NaN,Data Modeling,NaN
3,hmansell,Howard Mansell,NaN,howard@mansells.us,0,Facebook,https://avatars.githubusercontent.com/u/980875...,False,2016-08-24 15:41:21,30,...,False,False,True,False,0,{},2,New York,AI for Manufacturing,NaN
4,clementfarabet,Clement Farabet,NaN,clement.farabet@gmail.com,79,@NVIDIA,https://avatars.githubusercontent.com/u/188959...,False,2016-09-23 00:09:18,675,...,False,False,True,False,0,"{'C': 18, 'Lua': 45, 'Shell': 2, 'C++': 3, 'PH...",5,San Francisco,Data Management,NaN


In [89]:
df_tfidf = df[['bio']]
df_tfidf['id'] = df_tfidf.index
df_tfidf = df_tfidf[['id','bio']]
df_tfidf.head()

/var/folders/ml/8_bkc7k91cb777tm6mxkvfbc0000gn/T/ipykernel_42112/525773181.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tfidf['id'] = df_tfidf.index


,id,bio
0,0,NaN
1,1,Research Scientist
2,2,NaN
3,3,NaN
4,4,NaN


In [90]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df_tfidf['bio'].values.astype('U'))
tfidf_matrix.shape

(300, 1542)

In [91]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
results = {}

In [99]:
for idx, row in df_tfidf.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], df_tfidf['id'][i]) for i in similar_indices]
    results[row['id']] = similar_items[1:]

print('done!')

done!


In [100]:
def item(id):
    return df_tfidf.loc[df_tfidf['id'] == id]['bio'].tolist()[0].split(' - ')[0]

In [101]:
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")

recommend(item_id=11, num=5)

Recommending 5 products similar to Researcher @NVIDIA, Formerly Mila | CMU | MSR | FAIR | Element AI...
-------
Recommended: Research Scientist at FAIR
 (score:0.08801818621803285)
Recommended: Research Scientist in Meta AI Research (FAIR) (score:0.07802597074875811)
Recommended: Researcher Machine Learning and Vision at Concordia U/MILA/UdeM (score:0.07555146609620625)
Recommended: @scoutapp-ai  (score:0.06275448039732234)
Recommended: covariant.ai (score:0.06275448039732234)
